In [ ]:
!pip install torch lightning numpy kaggle wandb torch-geometric
!pip install polars  -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 35.5 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.17.3
    Uninstalling polars-0.17.3:
      Successfully uninstalled polars-0.17.3


In [ ]:
from google.colab import files

# Carica il file kaggle.json
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alexxxyy47","key":"f22a6402c7ed399e2946136b2e7223df"}'}

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"alexxxyy47","key":"f22a6402c7ed399e2946136b2e7223df"}'}

In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d ealaxi/paysim1
!unzip paysim1.zip
!rm paysim1.zip

100% 177M/178M [00:07<00:00, 33.9MB/s]
100% 178M/178M [00:07<00:00, 25.6MB/s]
Archive:  paysim1.zip
  inflating: PS_20174392719_1491204439457_log.csv  


In [ ]:
import pandas as pd, sys, plotly.graph_objects as go, plotly.express as px, numpy as np, torch, random as rnd, torch.nn as nn, lightning as l, wandb as wndb
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
from torch_geometric import seed_everything
import polars as pl
from torch_geometric.data import Data
import pdb
from torch_geometric.nn import GCNConv
import torchmetrics


In [ ]:
# PARAMETERS

DEVICE = "cuda"
SEED = 42

rnd.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
seed_everything(SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
ACCELERATOR =  "gpu" if torch.cuda.is_available() else "cpu"
POS_SIZE = 1200
NEG_SIZE = 150


In [ ]:
# UTILS FUNCTIONS

def load_dataframe( dataset_file : str):
    return pl.read_csv(dataset_file)


def find_null_or_empty_records( dataframe: pd.DataFrame):
    n = len(dataframe)
    for index, row in dataframe.iterrows():
        print_progress_bar(index/n)
        # Controlla se ci sono valori nulli o vuoti nel record
        if row.isnull().any() or any(map(lambda x: x == '', row)):
            # Stampa il record
            print(f"Record con valori nulli o vuoti:\n{row}\n")

def print_progress_bar(percentuale, lunghezza_barra=20):
    blocchi_compilati = int(lunghezza_barra * percentuale)
    barra = "[" + "=" * (blocchi_compilati - 1) + ">" + " " * (lunghezza_barra - blocchi_compilati) + "]"
    sys.stdout.write(f"\r{barra} {percentuale * 100:.2f}% completo")
    sys.stdout.flush()


def compute_kind_inconsistence(dataframe):
    return {"inconsistent orig balance": len(dataframe.query('abs(oldbalanceOrg - newbalanceOrig) != amount'))/len(dataframe),
            "inconsistent dest balance": len(dataframe.query('abs(oldbalanceDest - newbalanceDest) != amount'))/len(dataframe),
            "zero cash transaction": len(dataframe.query('amount == 0 '))/len(dataframe),
            "self-transaction": len(dataframe.query('nameOrig == nameDest'))/len(dataframe)
            }

def plot_histogram(to_plot):


    # Converti il dizionario in un array di valori
    values = list(to_plot.values())

    # Crea un istogramma
    fig = go.Figure(data=[go.Bar(x=list(to_plot.keys()), y=values)])

    # Mostra l'istogramma
    fig.show()



def plot_categories(dataframe):
    # Calcola la frequenza di ogni categoria nella colonna 'type'
    counts = dataframe['type'].value_counts().reset_index()

    # Rinomina le colonne
    counts.columns = ['type', 'count']

    counts['count'] = counts['count'] / counts['count'].sum()

    # Crea l'istogramma con Plotly Express
    fig = px.bar(counts, x='type', y='count', title='Istogramma delle categorie nella colonna "type"')

    # Mostra il plot
    fig.show()

def create_name_dict(df):
  df1 = df.select(pl.col("nameOrig").alias('name'))
  df2 = df.select(pl.col("nameDest").alias('name'))
  df = pl.concat([df1,df2])
  df = df.unique()
  names = list(df['name'])
  return dict(zip(names,list(range(len(names)))))


def divide_dataset(dataset_file,train_prc,val_prc):
  #breakpoint()
  dataframe = load_dataframe(dataset_file)
  transaction_types = {
      "CASH_IN": 0,
      "CASH_OUT": 1,
      "DEBIT": 2,
      "PAYMENT": 3,
      "TRANSFER": 4
  }

  dataframe = dataframe.with_columns(pl.col("type").replace(transaction_types).alias("type"),
                                     (pl.col('step')%24).alias('step'))

  id_df  = pl.DataFrame({'id': list(range(len(dataframe)))})

  dataframe = pl.concat([dataframe, id_df], how="horizontal")

  d_neg = dataframe.filter((pl.col('amount') != 0) & (pl.col('isFraud') == 0))
  neg_data_train = d_neg.sample(int(len(d_neg)*train_prc))
  d_neg = d_neg.filter(~pl.col('id').is_in(neg_data_train.select(pl.col('id'))))

  d_pos = dataframe.filter((pl.col('amount') != 0) & (pl.col('isFraud') == 1))
  pos_data_train = d_pos.sample(int(len(d_pos)*train_prc))
  d_pos = d_pos.filter(~pl.col('id').is_in(pos_data_train.select(pl.col('id'))))


  neg_data_val = d_neg.sample(int(len(d_neg)*val_prc))
  d_neg = d_neg.filter(~pl.col('id').is_in(neg_data_val.select(pl.col('id'))))

  pos_data_val = d_pos.sample(int(len(d_pos)*val_prc))
  d_pos = d_pos.filter(~pl.col('id').is_in(pos_data_val.select(pl.col('id'))))

  neg_data_train = neg_data_train.select(pl.exclude('id'))
  pos_data_train = pos_data_train.select(pl.exclude('id'))

  neg_data_val = neg_data_val.select(pl.exclude('id'))
  pos_data_val = pos_data_val.select(pl.exclude('id'))

  d_neg = d_neg.select(pl.exclude('id'))
  d_pos = d_pos.select(pl.exclude('id'))

  return (neg_data_train, pos_data_train), (neg_data_val, pos_data_val), (d_neg,d_pos )




In [ ]:
dataframe = load_dataframe("PS_20174392719_1491204439457_log.csv")
dataframe = dataframe.cast({"isFraud": pl.Int8})

In [ ]:
d = create_name_dict(dataframe)

In [ ]:
len(d.keys())

9073900

In [ ]:
divide_dataset("PS_20174392719_1491204439457_log.csv",0.7,0.1)

((shape: (4_448_084, 11)
  ┌──────┬──────┬───────────┬─────────────┬───┬──────────────┬──────────────┬─────────┬──────────────┐
  │ step ┆ type ┆ amount    ┆ nameOrig    ┆ … ┆ oldbalanceDe ┆ newbalanceDe ┆ isFraud ┆ isFlaggedFra │
  │ ---  ┆ ---  ┆ ---       ┆ ---         ┆   ┆ st           ┆ st           ┆ ---     ┆ ud           │
  │ i64  ┆ str  ┆ f64       ┆ str         ┆   ┆ ---          ┆ ---          ┆ i64     ┆ ---          │
  │      ┆      ┆           ┆             ┆   ┆ f64          ┆ f64          ┆         ┆ i64          │
  ╞══════╪══════╪═══════════╪═════════════╪═══╪══════════════╪══════════════╪═════════╪══════════════╡
  │ 16   ┆ 4    ┆ 2.7249e6  ┆ C111177078  ┆ … ┆ 0.0          ┆ 2.8515e6     ┆ 0       ┆ 0            │
  │ 15   ┆ 0    ┆ 122161.91 ┆ C348307229  ┆ … ┆ 2018844.6    ┆ 1.1433e6     ┆ 0       ┆ 0            │
  │ 16   ┆ 4    ┆ 115519.78 ┆ C106220047  ┆ … ┆ 0.0          ┆ 126625.6     ┆ 0       ┆ 0            │
  │ 20   ┆ 1    ┆ 268483.68 ┆ C1882402481 ┆ … ┆ 

In [ ]:
dataframe.columns


['step',
 'type',
 'amount',
 'nameOrig',
 'oldbalanceOrg',
 'newbalanceOrig',
 'nameDest',
 'oldbalanceDest',
 'newbalanceDest',
 'isFraud',
 'isFlaggedFraud']

In [ ]:
len(dataframe.filter(pl.col('amount') == 0))

16

In [ ]:
#| (pl.col('nameDest').str.starts_with('M'))   (abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig') )) == abs( pl.col('oldbalanceDest') - pl.col('newbalanceDest'))) |
print(len(dataframe.filter( (pl.col('nameDest').str.starts_with('M'))  )))
print(len(dataframe.filter( (pl.col('nameOrig').str.starts_with('M'))  )))
print(len(dataframe.filter( (pl.col('isFraud') == 1)  )))

2151495
0
8213


In [ ]:
print(len(dataframe.filter( (pl.col('nameDest').str.starts_with('M'))  |   (abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig') ) == abs( pl.col('oldbalanceDest') - pl.col('newbalanceDest')) )           )       ))

2393661


In [ ]:
print(len(dataframe.filter( (pl.col('isFraud') == 1) & (~pl.col('nameDest').str.starts_with('M'))  &   (abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig') ) != abs( pl.col('oldbalanceDest') - pl.col('newbalanceDest')) )           )       ))

6036


In [ ]:
df = pl.DataFrame({"a": [1, 2, 3], "b": [1, 2, 3] })

In [ ]:
print(df.sample(1))
print(df.sample(1))

shape: (1, 2)
┌─────┬─────┐
│ a   ┆ b   │
│ --- ┆ --- │
│ i64 ┆ i64 │
╞═════╪═════╡
│ 2   ┆ 2   │
└─────┴─────┘
shape: (1, 2)
┌─────┬─────┐
│ a   ┆ b   │
│ --- ┆ --- │
│ i64 ┆ i64 │
╞═════╪═════╡
│ 2   ┆ 2   │
└─────┴─────┘


In [ ]:
class FraudDetectionDataset(Dataset):

    def __init__(self,neg_data, pos_data,device):
      self.neg_data = neg_data
      self.pos_data = pos_data
      self.device = device



    def collate(self, pos_num, data ):
      breakpoint()
      pos = self.pos_data.sample_rows(pos_num)
      data = pl.concat([pos, data])
      name_d = create_name_dict(data)
      x = torch.tensor([1 if y.startswith("M") else 0 for y in name_d.keys()], dtype=torch.float).to(self.device)
      data = data.with_columns(pl.col('nameOrig').replace(name_d).alias('nameOrig'), pl.col('nameDest').replace(name_d).alias('nameDest'))
      edges = data.select(pl.col('nameOrig','nameDest'))
      edge_index = torch.tensor(edges[['nameOrig','nameDest']].to_numpy(), dtype=torch.long).t().contiguous().to(self.device)
      y = torch.tensor(data['isFraud'].to_numpy(), dtype=torch.float).to(self.device)
      edge_attr =  torch.tensor(data['amount'].to_numpy(), dtype=torch.float).to(self.device)
      data_graph = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
      return data, data_graph




    def __getitem__(self, index):
      return self.neg_data.iloc[index]

    def __len__(self):
      return len(self.neg_data)

    def get_dataloader(self, batch_size, pos_num):
      return DataLoader(self, batch_size=batch_size, shuffle=False, collate_fn = self.collate(pos_num))





In [ ]:
class FraudDetectionModule(nn.Module):

  def __init__(self,gnn_in_size, gnn_out_size, linear_in_size, linear_out_size, device):
    self.gnn = GCNConv(gnn_in_size,gnn_out_size)
    self.classifier = nn.Linear(linear_in_size, linear_out_size)
    self.sigmoid = nn.Sigmoid()
    self.relu = nn.ReLU()
    self.device = device

  def forward(self,data):
    edge_index = data[1].edge_index
    edge_attr = data[1].edge_attr
    x = data[1].x
    train_edges = data[0]


    train_features = torch.tensor(train_edges[['step','type','amount']].to_numpy(), dtype=torch.float ).to(self.device)

    x = self.relu(self.gnn(x, edge_index, edge_attr))

    from_nodes = torch.nan_to_num(x.squeeze()[edge_index[0,:].squeeze()])
    dest_nodes = torch.nan_to_num(x.squeeze()[edge_index[1,:].squeeze()])

    to_classify = torch.cat((from_nodes,dest_nodes,train_features), dim=1)

    out = self.classifier(to_classify)
    return out










In [ ]:
def train(model, epochs, train_dataloader, val_dataloader, loss, optimizer, f1):
  for epoch in range(epochs):
    # Addestramento
    model.train()
    train_loss_epoch = []
    for batch_inputs in train_dataloader:
        outputs = model(batch_inputs)
        train_loss = loss(outputs, batch_inputs[1].y)
        train_loss_epoch.append(train_loss)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

    val_loss, f1_score = validate(model, val_dataloader,loss,f1)
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss_epoch.mean()}, Validation Loss: {val_loss}, f1 score = {f1_score}')




def validate(model, dataloader, loss, f1):
  model.eval()
  with torch.no_grad():
    val_loss_out = []
    f1_out = []
    for batch_inputs in dataloader:
      val_outputs = model(batch_inputs)
      val_loss = loss(val_outputs, batch_inputs[1].y)
      val_f1 = f1(val_outputs, batch_inputs[1].y)
      f1_out.append(val_f1)
      val_loss_out.append(val_loss)
  return val_loss_out.mean(), f1_out.mean()


In [ ]:
train_set, validation_set, test_set = divide_dataset("PS_20174392719_1491204439457_log.csv",0.7,0.1)

train_dataset =  FraudDetectionDataset(train_set[0], train_set[1], DEVICE)
validation_dataset =  FraudDetectionDataset(validation_set[0], validation_set[1], DEVICE)

train_loader = train_dataset.get_dataloader(NEG_SIZE,POS_SIZE)
validation_loader = validation_dataset.get_dataloader(NEG_SIZE,POS_SIZE)

In [ ]:
model = FraudDetectionModule(1,8,19,1,DEVICE).to(DEVICE)

In [ ]:
train(model,30,train_loader,validation_loader,nn.BCEWithLogitsLoss(), torch.optim.Adam(model.parameters(), lr=1e-3), torchmetrics.classification.BinaryF1Score() )

In [ ]:
d = compute_kind_inconsistence(inconsistent_data)
plot_histogram(d)
names_distinct = set(dataset.raw_data["nameOrig"]).union(set(dataset.raw_data["nameDest"]))